In [ ]:
experiment = 'iw_loss.yaml'
%run -i ../template.ipynb

In [ ]:
body

In [ ]:
def list_of_runs(input_list):
    runs = []
    for k in input_list:
        run = k.split('_')[1]
        if run not in runs:
            runs.append(run)
    return runs 

In [ ]:
runs = os.listdir(netsdir)

runs = [x for x in runs if prefix in x]
loruns = list_of_runs(runs)  # to iterate over

print(len(loruns))

In [ ]:
prefix

In [ ]:
name = loruns[70]
wildcard = prefix + '_' + name + '*'
dist_dicts, info_dicts, loss_dicts, net_dicts, posterior_dicts, sim_dicts = io.load_prefix(netsdir, '*074669aa*')

In [ ]:
len(net_dicts)

In [ ]:
net = io.nth(net_dicts, 0)

In [ ]:
p = net.get_params()

In [ ]:
p.keys()

In [ ]:
for key in p.keys():
    print(np.var(p[key]))

In [ ]:
nth(posterior_dicts,1).xs[0].S

In [ ]:
nth(info_dicts,1)

In [ ]:
to = len(nth(loss_dicts, 1)['trn_out'])

In [ ]:
w = []
for t in range(to):
    weights = nth(loss_dicts, 1)['trn_out'][t][1]
    wm = np.sum(np.log(weights))
    w.append(wm)

In [ ]:
plt.plot(nth(loss_dicts, 0)['val_val'])

In [ ]:
plt.plot(nth(loss_dicts, 3)['trn_val'])
#plt.ylim([-1,4])
plt.xlim([1000, 2500])

In [ ]:
df = pd.DataFrame()

for r in tqdm(range(len(loruns))):
    name = loruns[r]
    wildcard = prefix + '_' + name + '*'
    
    path = netsdir
    #dist_dicts = io.load_wildcard(path, wildcard + '_dist')
    info_dicts = io.load_wildcard(path, wildcard + '_info')
    
    info = last(info_dicts)
    #if int(nth(info_dicts, 3)['datetime'][11:13]) < 20:
    #    continue
        
    #loss_dicts = io.load_wildcard(path, wildcard + '_loss')
    #net_dicts = io.load_wildcard(path, wildcard + '_net')
    posterior_dicts = io.load_wildcard(path, wildcard + '_posterior')
    sim_dicts = io.load_wildcard(path, wildcard + '_sim')
    

        
    sim = last(sim_dicts)
    approx_posterior = last(posterior_dicts)
    
    try:

        posterior_mu = approx_posterior.xs[0].m[0]
        posterior_cov = approx_posterior.xs[0].S[0,0]

        true_mu = sim.posterior.m[0]
        true_cov = sim.posterior.S[0,0]

        std1 = np.sqrt(true_cov)
        std2 = np.sqrt(posterior_cov)
        mu1 = true_mu
        mu2 = posterior_mu
        kltp = np.log(std2/std1) + (std1**2 + (mu1-mu2)**2)/(2*std2**2) - 0.5

        iw_loss = info['iw_loss']
        svi = info['svi']
        prior_alpha = info['prior_alpha']
        prior_true = type(nth(info_dicts, -1)['prior_true'])
        prior_true = 'Gaussian' in str(prior_true)

        dim = nth(sim_dicts, -1).dim

        n_summary = last(sim_dicts).n_summary
        #n_samples = len(last(dist_dicts))

        seed = first(info_dicts)['seed']

        data = [
                    [name],
                    [seed],
                    [dim],
                    [iw_loss],
                    [svi],
                    [prior_alpha],
                    [prior_true],
                    [posterior_mu],
                    [posterior_cov],
                    [true_mu],
                    [true_cov],
                    [kltp],
                    [info['reg_scale_iw']],
                    [info['keep_n']],
                    [info['ess']],
                    [info['ess_lc']],
                    [info['loss_calib']],
                    [info['normalize_weights']]
                ]

        data = pd.DataFrame(data).T
        data.columns = ['name', 'seed', 'dim', 'iw_loss', 'svi', 'prior_alpha', 'prior_true', 'p_mu', 'p_cov', 
                        't_mu', 't_cov', 'kltp', 'rsiw','keep','ess','ess_lc','loss_calib', 'norm']

        df = df.append(data)    
    except:
        pass
        
df['abserr_cov_tp'] = np.abs(df['t_cov'] - df['p_cov'])
df['abserr_mu_tp'] = np.abs(df['t_mu'] - df['p_mu'])

df = df.reset_index()
df = df.drop('index', 1)

print(len(df))
df.head()

df.to_pickle('iw_loss.pkl')

In [ ]:
for r in range(len(df['loss_calib'])):
    if df.loc[r, 'loss_calib'] is None:
        df.loc[r, 'loss_calib'] = 0.
    else:
        df.loc[r, 'loss_calib'] = 1.

In [ ]:
df.to_pickle('iw_loss.pkl')

In [ ]:
df = pd.read_pickle('iw_loss.pkl')

In [ ]:
#dfa = df.copy()
dfc = df.copy()

In [ ]:
subset = dfc.query('ess == True').copy()
subset

In [ ]:
subset = dfc.query('dim == 1 & iw_loss == True & svi == True & prior_true == False').copy()
subset = subset.reset_index()
subset = subset.drop('index', 1)

sns.set(font_scale=2)
g = sns.factorplot(x="prior_alpha", y="p_mu", data=subset,
                   capsize=.1, palette="YlGnBu_d", size=10, aspect=1., markers='.',
                   linestyles='-', legend=False, col="ess", row="keep", sharey=True)
ax = plt.gca()
plt.legend()
#ax.set_ylim([0, 0.012])
#ax.set_ylim([0, 0.05])
#ax.semilogy()
#saverep('loss_calib')

In [ ]:
dfc

In [ ]:
dfc.head()

In [ ]:
subset = dfc.query('dim == 1 & iw_loss == False & svi == True') # & loss_calib == 1 & ess == True & keep == True & norm == True & ess_lc == True').copy()
subset = subset.reset_index()
subset = subset.drop('index', 1)

sns.set(font_scale=2)
g = sns.factorplot(x="prior_alpha", y="abserr_cov_tp", data=subset,
                   capsize=.1, palette="YlGnBu_d", size=10, aspect=1., markers='.',
                   linestyles='-', legend=False, sharey=False)
ax = plt.gca()
#ax.set_ylim([0, 0.012])
#ax.set_ylim([0, 0.02])
#ax.semilogy()
#plt.legend()
#saverep('loss_calib')

In [ ]:

subset

In [ ]:
subset

In [ ]:
subset = dfc.query('dim == 1 & iw_loss == False & svi == False').copy()
subset = subset.reset_index()
subset = subset.drop('index', 1)

plt.hist(subset['abserr_cov_tp'])

In [ ]:
subset = dfc.query('dim == 1 & iw_loss == False & svi == True').copy()
subset = subset.reset_index()
subset = subset.drop('index', 1)

plt.hist(subset['abserr_cov_tp'])